In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
# make char with Char RNN
# make vocabulary
input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str+label_str)))
vocab_size = len(char_vocab)
print ('문자 집합의 크기 : {}'.format(vocab_size))
# one-hot vector
input_size = vocab_size # 입력의 크기는 문자 집합의 크기
hidden_size = 5
output_size = 5
learning_rate = 0.1
# word embedding
char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 문자에 고유한 정수 인덱스 부여
print(char_to_index)
index_to_char={}
for key, value in char_to_index.items():
    index_to_char[value] = key
print(index_to_char)
# each word map with integer
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data)
print(y_data)
# convert to 3D tensor
x_data = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)
# convert to one-hot vector
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)
# convert to tensor
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))
# RNN model
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True) # RNN 셀 구현
        self.fc = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층 구현
    def forward(self, x): # 구현한 RNN 셀과 출력층을 연결
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x
# get model
net = Net(input_size, hidden_size, output_size)
# output
outputs = net(X)
print(outputs.shape) # 3차원 텐서
print(outputs.view(-1, input_size).shape) # 2차원 텐서로 변환
print(Y.shape)
print(Y.view(-1).shape)
# cost function
criterion = torch.nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)
# training
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1)) # view를 하는 이유는 Batch 차원 제거를 위해
    loss.backward() # 기울기 계산
    optimizer.step() # 아까 optimizer 선언 시 넣어둔 파라미터 업데이트
    # 아래 세 줄은 모델이 실제 어떻게 예측했는지를 확인하기 위한 코드.
    result = outputs.data.numpy().argmax(axis=2) # 최종 예측값인 각 time-step 별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    if i % 5 == 0:
        print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

문자 집합의 크기 : 5
{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}
{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}
[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]
[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]
[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]
훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([1, 5])
torch.Size([1, 5, 5])
torch.Size([5, 5])
torch.Size([1, 5])
torch.Size([5])
0 loss:  1.8460521697998047 prediction:  [[1 1 1 1 1]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  aaaaa
5 loss:  0.7908139228820801 prediction:  [[4 4 3 4 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pplp!
10 loss:  0.2766772210597992 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
15 loss:  0.07700546830892563 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
20 loss:  0.01569977030158043 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
25 loss:  0.005719

/var/folders/v5/42kgxl691jx5h0jzyx8czt_w0000gn/T/ipykernel_5628/1784842044.py:34: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1634272482218/work/torch/csrc/utils/tensor_new.cpp:201.)
  X = torch.FloatTensor(x_one_hot)


In [3]:
# make sentense with char RNN
# preprocessing train data
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")
char_set = list(set(sentence)) # 중복을 제거한 문자 집합 생성
char_dic = {c: i for i, c in enumerate(char_set)} # 각 문자에 정수 인코딩
dic_size = len(char_dic)
# set hyperparameter
hidden_size = dic_size
sequence_length = 10  # 임의 숫자 지정
learning_rate = 0.1
# slice sample for train data
x_data = []
y_data = []
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    x_data.append([char_dic[c] for c in x_str])  # x str to index
    y_data.append([char_dic[c] for c in y_str])  # y str to index
# one-hot vector
x_one_hot = [np.eye(dic_size)[x] for x in x_data] # x 데이터는 원-핫 인코딩
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))
# RNN model
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers): # 현재 hidden_size는 dic_size와 같음.
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)
    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x
# get model
net = Net(dic_size, hidden_size, 2) # 이번에는 층을 두 개 쌓습니다.
# cost function
criterion = torch.nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)
outputs = net(X)
print(outputs.shape) # 3차원 텐서
print(outputs.view(-1, dic_size).shape) # 2차원 텐서로 변환.
print(Y.shape)
print(Y.view(-1).shape)
# training
for i in range(1001):
    optimizer.zero_grad()
    outputs = net(X) # (170, 10, 25) 크기를 가진 텐서를 매 에포크마다 모델의 입력으로 사용
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()
    # results의 텐서 크기는 (170, 10)
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오지만
            predict_str += ''.join([char_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += char_set[result[-1]]
    if i % 100 == 0:
        print(i, "loss: ", loss.item(), "prediction str: ", predict_str)

훈련 데이터의 크기 : torch.Size([170, 10, 25])
레이블의 크기 : torch.Size([170, 10])
torch.Size([170, 10, 25])
torch.Size([1700, 25])
torch.Size([170, 10])
torch.Size([1700])
0 loss:  3.204636335372925 prediction str:  ofolfloofofofflofllfooooflfl fllofoffllofllflllloooflofflloffloflooffolflfooofofflofoloooooooflloffoflooofolflloooffoflofllofflofooflloffloflooofloofloolololoooffllooflofloffloofl
100 loss:  0.2410643994808197 prediction str:  l you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
200 loss:  0.23215584456920624 prediction str:  g you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
300 loss:  0.23064179718494415 prediction str:  g you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, 

In [6]:
# make sentense with word embedding RNN
# preprocessing
# sample data split by word
sentence = "Repeat is the best medicine for memory".split()
vocab = list(set(sentence))
# word map index
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}  # 단어에 고유한 정수 부여
word2index['<unk>']=0
index2word = {v: k for k, v in word2index.items()}
# make input & label data
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 각 문자를 정수로 변환. 
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    return input_seq, label_seq
# get data
X, Y = build_data(sentence, word2index)
# RNN model
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, # 워드 임베딩
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, # 입력 차원, 은닉 상태의 크기 정의
                                batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) # 출력은 원-핫 벡터의 크기를 가져야함. 또는 단어 집합의 크기만큼 가져야함.
    def forward(self, x):
        # 1. 임베딩 층
        # 크기변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        # 크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))
# hyperparameter
vocab_size = len(word2index)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다. <unk> 토큰을 크기에 포함한다.
input_size = 5  # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20  # RNN의 은닉층 크기
# get model
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
# cost function
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안 해도 됨.
# optimizer
optimizer = optim.Adam(params=model.parameters())
# test process
output = model(X)
print(output.shape)
# for confirm prediction correct
decode = lambda y: [index2word.get(x) for x in y]
# training
for step in range(301):
    # 경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(X)
    # 손실값 계산
    loss = loss_function(output, Y.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()
    # 기록
    if step % 100 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))

torch.Size([6, 8])
[01/201] 2.0123 
Repeat Repeat is for Repeat for for
[101/201] 0.5903 
Repeat is the best medicine for memory
[201/201] 0.1399 
Repeat is the best medicine for memory
[301/201] 0.0593 
Repeat is the best medicine for memory
